In [1]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import math
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *

In [2]:
class Node():
    def __init__(self,node):    
        self.node = node
    def invert(self):        
        return node_df.loc[self.node,'Invert Level']
    def ground(self):        
        return node_df.loc[self.node,'Ground Level']
    def soh(self):        
        return node_df.loc[self.node,'Safe Operating Head']
    def asset_name(self):        
        return node_df.loc[self.node,'Asset Name']
    def cover(self):        
        return node_df.loc[self.node,'Cover Type']

class Pipe():
    def __init__(self,pipe):    
        self.pipe = pipe
    def height(self):        
        return pipe_df.loc[self.pipe,'Height'] 
    def uplevel(self):
        uplevel = pipe_df.loc[self.pipe,'UpLevel']
        if math.isnan(uplevel):
            node = pipe_df.loc[self.pipe,'FromNode']
            uplevel = node_df.loc[node,'Invert Level']
        return uplevel
    def dwlevel(self):
        dwlevel = pipe_df.loc[self.pipe,'DwLevel']
        if math.isnan(dwlevel):
            node = pipe_df.loc[self.pipe,'ToNode']
            dwlevel = node_df.loc[node,'Invert Level']
        return dwlevel
    

In [3]:
for m in master_list:
    
    result_folder = m[2]

    result_list = m[4]
    
    node_df = pd.read_csv(result_folder + '\\ls_nodes.csv',dtype={'Node': str})
    node_df.set_index('Node',inplace=True)
    node_df['Asset Name'].fillna('',inplace=True)
    pipe_df = pd.read_csv(result_folder + '\\ls_pipes.csv',dtype={'Pipe': str,'FromNode': str,'ToNode': str})
    pipe_df.set_index('Pipe',inplace=True)
    
    for r in result_list:
        header = r[0]
        file = r[1]
        res1d = Res1D(result_folder + '\\' + file)
        r.append(res1d)


In [4]:
missing_pipes = []

node_delta_xs = [['US',-1],['DS',0]]

acronyms = list(pipe_df.Acronym.unique())
acronyms.sort()
paths = []

loop_counter = 0

for acronym in acronyms:
    print(acronym)
    
    path_no = 0
    
    pipe_index_df = pipe_df[pipe_df.Acronym==acronym].copy()
    
    while len(pipe_index_df)>0:
             

        start_pipes = []
        for index, row in pipe_index_df.iterrows():
            if row['FromNode'] not in list(pipe_index_df.ToNode):

                start_pipes.append(index)

        start_pipes

        for start_pipe in start_pipes[:1]:
            exit_path = False
            chainage = 0

            node = pipe_index_df.loc[start_pipe,'FromNode']
            pipe = start_pipe

            path = []
            
            for node_delta_x in node_delta_xs:
                point = [path_no,0,acronym,'Node-'+node_delta_x[0],node,chainage+node_delta_x[1],Node(node).invert(),
                         Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                        np.nan,np.nan]
                for r in result_list:
                    point.append(max(r[3].query.GetNodeValues(node, "WaterLevel")))
                path.append(point)

            i = 0
            while exit_path == False and i < 1000:

                point = [path_no,i*3+1,acronym,'Pipe US',pipe,chainage,Pipe(pipe).uplevel(),
                             Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                            Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).uplevel()]

                for r in result_list:
                    point.append(max(r[3].query.GetReachStartValues(pipe, "WaterLevel")))
                path.append(point)

                chainage += pipe_index_df.loc[pipe,'Length']

                node = pipe_index_df.loc[pipe,'ToNode']


                point = [path_no,i*3+1,acronym,'Pipe DS',pipe,chainage-1,Pipe(pipe).dwlevel(),
                             Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                            Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).dwlevel()]

                for r in result_list:
                    point.append(max(r[3].query.GetReachEndValues(pipe, "WaterLevel")))
                
                path.append(point)
                

                for node_delta_x in node_delta_xs:
                    point = [path_no,0,acronym,'Node-'+node_delta_x[0],node,chainage+node_delta_x[1],Node(node).invert(),
                                 Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                                np.nan,np.nan]

                    for r in result_list:
                        point.append(max(r[3].query.GetNodeValues(node, "WaterLevel")))
                

                    path.append(point)

                pipe_index_df.drop(pipe,inplace=True)

                try:
                    pipe = pipe_index_df[pipe_index_df.FromNode==node].index[0]
                except:
                    exit_path = True   
                i += 1
                
                loop_counter += 1
                if loop_counter > 20000:
                    raise
                    
                
        paths += path
        
        path_no += 1

# paths
columns = ['Path Number','Sqn','Acronym','Type','MUID','Chainage','Invert Level','Ground Level','Safe Operating Head',
                                   'Asset Name','Cover Type','Height','Pipe Obvert']

for r in result_list:
    header = r[0]
    columns.append(header)
    

path_df=pd.DataFrame(paths,columns=columns)
path_df.replace(999,np.nan,inplace=True)
       

FAT
GE2
GE3
GN2C
GN2H
GNE
GNG
GNH
GNK
GNM
HLK
HLL
HLR
HLY
KEO
LNG1
LYN
MKO
MKT
NVC
NVD
NVF
NVL
NVLS
NVLS2
NVLS3
NVS
NVT
NVY
NVY2
TWE
TWT


In [7]:
# with open(r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Plot_Experiments\missing.mus', 'w') as f:
#     f.write('msm_Link\n')
#     for pipe in pipe_df.index:
#         if not pipe in  path_df[path_df.Type!='Node'].MUID.unique():
#             f.write(str(pipe) + '\n')
# f.close()
path_df.replace(999,np.nan,inplace=True)

In [8]:
# output_folder = r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Plots'

colors = ['black','black','green','red','brown','orange','grey','brown','midnightblue','tan','firebrick','darkturquoise',
         'darkorchid','dimgray','darkgoldenrod','indianred','deeppink']



for j, acronym in enumerate(acronyms):
    
    
    path_acronym_df = path_df[path_df.Acronym==acronym]
    
    
    
    output_subfolders = [output_folder + "\\All_Longsections"]
    if groupby_acronym_owner == True:
        output_subfolders.append(output_folder + '\\GV_Acronym_' + acronym)

    for i, output_subfolder in enumerate(output_subfolders):
        if not os.path.isdir(output_subfolder): os.makedirs(output_subfolder) 

        with open(output_subfolder + "\\Long_Section_" + acronym + ".html", 'w') as f:
            f.write('<link rel="stylesheet" href="..\Maps_And_CSS\style.css">\n')

            f.write('<!DOCTYPE html>\n')
            f.write('<html>\n')
            f.write('<head>\n')
            f.write('<meta charset="utf-8">\n')
            f.write('</head>\n')
            f.write('<body>\n')
            f.write('<div class="sidenav">\n')

            
            
            f.write("<h2>Links to Others (Alphabetic Order)</h2>\n")
            if j == 0:
                previous_acronym = acronyms[len(acronyms)-1]
            else:
                previous_acronym = acronyms[j-1]
            if j == len(acronyms)-1:
                next_acronym = acronyms[0]
            else:
                next_acronym = next_acronym = acronyms[j+1]
            
            if i == 0:
                path = output_subfolder +  "\\Long_Section_" + next_acronym + ".html"
            else:                
                path = output_folder +  '\\GV_Acronym_' + next_acronym + "\\Long_Section_" + next_acronym + ".html"
                
            f.write('<a href="' + path + '">Next (' + next_acronym + ')</a>\n')
            f.write("<br>")
            
            if i == 0:
                path = output_subfolder +  "\\Long_Section_" + previous_acronym + ".html"
            else:                
                path = output_folder +  '\\GV_Acronym_' + previous_acronym + "\\Long_Section_" + previous_acronym + ".html"
                
            f.write('<a href="' + path + '">Previous (' + previous_acronym + ')</a>\n')
            f.write("<br>")

            f.write("<h2>Map</h2>\n")
            
            map_string = '<img src="..\\Maps_And_CSS\\Acronym_' + acronym + '.jpg" alt="' + acronym + '">\n'
            f.write(map_string + "\n")
            f.write("<p></p>\n")

            f.write('</div>')
            f.write('<div class="main">\n')
            f.write("<h1>Long Sections for " + acronym + "</h1>\n")

            for path_no in path_acronym_df['Path Number'].unique():
                path_acronym_sub_df = path_acronym_df[path_acronym_df['Path Number']==path_no]
                path_acronym_node_df = path_acronym_sub_df[path_acronym_sub_df.Type=='Node-DS']

                fig = go.Figure()

                columns=['Invert Level','Pipe Obvert','Ground Level','Safe Operating Head']
                for r in result_list:
                    header = r[0]
                    columns.append(header)

                for i, column in enumerate(columns):
                    if i < 4:
                        line_width = 4
                    else:
                        line_width = 2
                    fig.add_trace(go.Scatter(x=path_acronym_sub_df.Chainage, 
                                             y = path_acronym_sub_df[column],line_color=colors[i],name=column, line_width = line_width))

                lbl_x = []
                lbl_y = []
                lbl_text = []

                label_y = max(path_acronym_node_df['Ground Level']) + 2

                for index, row in path_acronym_node_df.iterrows():

                    fig.add_shape(type="rect",x0=row['Chainage']-1, y0=row['Invert Level'], x1=row['Chainage'],
                                  y1=row['Ground Level'],line=dict(color="RoyalBlue",width=1),)

                    fig.add_shape(type="line",
                        x0=row['Chainage']-0.5, y0=row['Ground Level'], x1=row['Chainage']-0.5, y1=label_y-0.8,
                        line=dict(
                            color="Grey",
                            width=1,
                            dash="dot",
                        )
                    )

                    lbl_x.append(row['Chainage'] - 0.5)

                    lbl_y.append(label_y)
                    if row['Cover Type']==2:
                        cover_type = 'Sealed'
                    else:
                        cover_type = 'Unsealed'
                    lbl_text.append(str(row['Asset Name']) + '<br>' +  str(row['MUID']) + '<br>' + cover_type)

                fig.add_trace(go.Scatter(
                    x=lbl_x,
                    y=lbl_y,
                    text=lbl_text,
                    mode="text",
                    name="Labels"
                ))

                fig.update_layout(
                    autosize=False,
                    width = 1362,
                    height=700,
                    margin=dict(
                        l=50,
                        r=50,
                        b=25,
                        t=35,
                        pad=4
                        ),
                    yaxis_title='Geodetic Elevation(m)',
                    xaxis_title='Chainage (m)'
                    )

                f.write("<h2>Long Section " + str(path_no+1) + "</h2>\n")
                f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

            f.write('</div>')
            f.write('</body>')
            f.write('</html>')
            f.close()